In [213]:
!pip3 install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [278]:
import sys
sys.path.append('src/')
import pandas as pd
from sklearn.model_selection import train_test_split
from Helpers import *
from sklearn.metrics import f1_score


# panda view settings
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns

In [215]:
df = pd.read_csv("./data/toxicbias_train.csv")
df_bias = df[df['bias'] == 'bias']
df_neutral = df[df['bias'] == 'neutral']


## Split up dataframe by category

In [216]:
# store unique categories
unique_categories = set()

# Iterate through each entry in the 'category' column
for categories in df['category'].str.split(','):
    for category in categories:
        stripped_category = category.strip()
        if stripped_category and stripped_category.lower() != 'none':
            unique_categories.add(stripped_category)

unique_categories_list = sorted(list(unique_categories))

In [217]:
categorical_dfs = {category: pd.DataFrame(columns=df.columns) for category in unique_categories_list}

# Split categories and add rows to the corresponding dataframes in dictionary
def split_categories_and_add_rows(row):
    categories = row['category'].split(',')
    for category in categories:
        category = category.strip()  # Remove leading/trailing spaces
        if category in categorical_dfs:
            categorical_dfs[category] = categorical_dfs[category].append(row, ignore_index=True)

df_bias.apply(split_categories_and_add_rows, axis=1)

# Print the shape of each category dataframe
for category, category_df in categorical_dfs.items():
    print(f"Dataframe for {category}:")
    print(f"Shape: {category_df.shape}")
    print("\n")

Dataframe for gender:
Shape: (293, 6)


Dataframe for lgbtq:
Shape: (453, 6)


Dataframe for political:
Shape: (172, 6)


Dataframe for race:
Shape: (1768, 6)


Dataframe for religion:
Shape: (1257, 6)




## Split training and test data

In [218]:
training_data = {}
test_data = {}

# Perform the 80-20 split for each category dataframe
for category, df in categorical_dfs.items():
    train_df, test_df = train_test_split(df, test_size=0.8, random_state=42)
    
    training_data[category] = train_df
    test_data[category] = test_df

In [219]:
training_data['political']

,comment_text,identity_attack_x,bias,target,category,rationale
143,Just another state ruled by Republican heavily governmentsubsidized farmers ranchers oil and gas living on their giant estates voting against all taxation while sucking up the government cash. Fundamentalist christian theocrats ruling over every city county and state governing system as they ban abortions defund all social programs and maintain the working classes in indentured servitude. All while proclaiming how friendly they all are except of course if you are nonwhite or refuse to beli...,0.500000,bias,conservatives,political,prejudice against conservativess and bigots
157,You forgot rightwing nuts Republican racists multinational corporate facists company men the Suits fraternity alums private security mercenaries evangelicals Dominionists and C average lawyers low education rednecks. Maybe thats who you meant when you wrote ...and hypocrites??,0.277778,bias,republicans,political,prejudice against republicans
63,LW1. Im 30 and I dont drink do drugs or fall for the feminist crap that modern women are a part of. I also dont appreciate that you put all young women in this nutty modern woman category.,0.528571,bias,feminists,political,prejudice against whites
54,Thats like saying the conservatives and GOP have endlessly supported racism and bigotry for the last four decades but suddenly want everyone to tolerate and embrace white supremacist. See how ridiculous that sounds.,0.500000,bias,"conservatives,whites","political,race",prejudice against conservativess
107,Agreed on most points but... Really I want liberals and progressives to murder their babies. I dont think they should have any children. They are diseased. They shouldnt be allowed anywhere near children.,0.262500,bias,"liberals,progressives",political,prejudice against liberals and progressives
50,When you lay down with dogs you wake up with fleas the old saying says. This rightwing absolutist Christian would have little appeal to thinking people. But American isAmerica.,0.500000,bias,"right wing,christians","political,religion",prejudice against right wing christians
169,Its official. Canadians have been driven to mental illness.,0.621622,bias,canadians,political,prejudice against canadians
58,Any suggestions or are you blowing smoke. You and all your liberal retards are all the same.,0.262295,bias,liberals,political,prejudice against liberals
48,The reality is that nazis neonazis white supremacists are nothing but scum. The confederate traitors lost get over it.,0.585714,bias,"nazis,whites","political,race",nazis and whites are scums
88,Yep black people. That is why they are uspet. Am y of those weirdoes gaslight the protesters saying thst what they feel and protest are not real. It is what abusive people do to keep their significant others close. I shade conservative but I have zero problem with protest. I wonder why these red blooded Americans support Trump who hates veterans and war heroes and is a draft dodger and a coward. I guess they are hypocrites and hopelessly stupid.,0.514286,bias,conservatives,political,prejudice against conservatives


## Premises

In [220]:
hypotheses = {
  'gender' : [
    'A person belonging to a gender lacks intelligence.',
    'A person belonging to a gender is a criminal.',
    'A person belonging to a gender is violent.',
    'Violence is encouraged against a gender.',
    'A person belonging to a gender is abusive.',
    'A person belonging to a gender should be blamed.',
    'Women make decisions based on emotions.',
    'A gender is unable to do some jobs.',
    'A gender is unable to do something as well as another gender.',
    'A gender should not talk.',
    'A person belonging to a gender is compared to an object.',
    'A gender should be blamed.',
    'A person belonging to a gender has an advantage compared to others.',
    'A person belonging to a gender is misogynist.',
    'A person belonging to a gender is disliked.',
  ],
  'lgbtq' : [
    'Lgbtq persons are believed to have a condition.',
    'Lgbtq should not be part of society.',
    'Violence is encouraged against lgbtq persons.',
    'Lgbtq persons are compared to animals.',
    'Lgbtq persons are believed to have an agenda.',
    'Lgbtq persons should be excluded.',
    'Lgbtq persons should stay home.',
    'Lgbtq persons commit crimes.',
    'Lgbtq persons get preferred to other persons.',
    'Lgbtq persons are seen as not being normal.', 
    'Lgbtq persons should not be allowed to marry.', 
    'Lgbtq persons have a disease.',
    'Lgbtq persons are seen as being disgusting.',
    'Lgbtq persons are not tolerated or desired.',
    'Lgbtq persons are treated differently.'

  ],
  'political' : [ 
      'A person belonging to a political group is stupid.',
      'A person belonging to a political group is crazy.',
      'Things are worse due to a political group.',
      'Members of a political group are lgbtq.',
      'Members of a political group are hypocritical.',
      'Members of a political group dislike a gender or lgbtq.',
      'Members of a political group commit crimes.',
      'Members of a political group are violent.',
      'Members of a political group are hateful.',
      'Members of a political group are compared to Nazis.',
      'Members of a political group are racist.',
      'A person belonging to a political group has a disease.',
      'Members of a political group are taking control.',
      'Members of a political group prefer minorities.',
      'Members of a political group hate their country or race.',
  ],
  'race' : [
    'Members of a race commit crimes.',
    'Members of a race are violent.',
    'Members of a race are compared to animals.',
    'The use of violence is encouraged or desired against a race.',
    'Members of a race should remain in their respective countries.',
    'The integration of a race should be avoided in a country or society.',
    'Members of a race are a threat to others.',
    'A race is inferior when compared to another race.',
    'A race is preferred to another.',
    'Members of a race are stupid.',
    'There are too many members of a race.',
    'Members of a race should be excluded.',
    'Members of a race should go back to their homes.',
    'Members of a race are refused.',
    'Members of a race are racist.'
  ],
  'religion' : [
    'The use of violence is encouraged or desired against a religion.',
    'A religion should be discriminated against.',
    'Members of a religion should be excluded and marginalized.',
    'Members of a religion should remain in their respective countries.',
    'Members of a religion are compared to animals.',
    'A religion or members of a religion are compared to devils or demons.',
    'The integration of a religion should be avoided in a country or society.',
    'A religion or members of a religion are stupid.',
    'A religion is inferior when compared to another religion.',
    'A religion is preferred to another.'
    'Religious practices of a particular faith should be restricted or banned in public spaces.',
    'Religious attire or symbols should be prohibited.',
    'A religion promotes violence or intolerance.',
    'Members of a religion are a threat.',
    'Members of a religion are not trustworthy or loyal.'
  ],
}

## Add neutral cases

In [221]:
def add_rows_to_dataframes(dataframes_dict, new_dataframe):

    for key, df in dataframes_dict.items():
        # Calculate the maximum number of rows to add
        max_rows_to_add = min(len(new_dataframe), len(df))

        # Choose random rows from the new dataframe
        rows_to_add = new_dataframe.sample(n=max_rows_to_add, replace=True)

        rows_to_add = new_dataframe.iloc[:max_rows_to_add]
        df = pd.concat([df, rows_to_add], ignore_index=True)
        dataframes_dict[key] = df


In [223]:
add_rows_to_dataframes(test_data, df_neutral)

## Bart model

In [254]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def bart_nli_batched_hypothese(premises, hypotheses):
    model_name = 'facebook/bart-large-mnli'
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model.to(DEVICE) 

    num_premises = len(premises)
    num_hypotheses = len(hypotheses)
    results = {}

    for i in range(num_premises):
        labels = []

        # Tokenize the input pair
        inputs = tokenizer([premises[i]] * num_hypotheses, hypotheses, return_tensors='pt', padding=True, truncation=True)

        # Run the input through the model
        logits = model(**inputs.to(DEVICE)).logits

        # Get probabilities and labels for the input
        probabilities = torch.softmax(logits, dim=1)
        label_mapping = ['contradiction', 'neutral', 'entailment']
        labels = [label_mapping[p.argmax().item()] for p in probabilities]

        results[premises[i]] = (labels, convert_probabilities_batched(probabilities, label_mapping))
    return results


## Run data through model and add labels and probabilities

In [267]:
def run_bart_model_add_labels(category):
    # Create a copy of the input dataframe
    df_copy = categorical_dfs[category].copy()

    # Run BART NLI on the dataframes for the specified category
    results = bart_nli_batched_hypothese(list(df_copy['comment_text']), hypotheses[category])

    # Process the results separately and add them to the dataframe
    labels_column = []
    probabilities_column = []

    for premise in df_copy['comment_text']:
        if premise in results:
            labels, probabilities = results[premise]
        else:
            labels, probabilities = None, None
        labels_column.append(labels)
        probabilities_column.append(probabilities)

    df_copy['labels'] = labels_column
    df_copy['probabilities'] = probabilities_column

    # sum_of_labels_nan = df_copy.loc[df_copy['labels'].isna(), 'labels'].sum()
    # print(sum_of_labels_nan)
    
    # # Filter out rows where 'labels' column has NaN values
    # filtered_df = df_copy.dropna(subset=['labels'])

    return df_copy

In [283]:
def determine_label_accuracy(df):
    def is_correct_bias(row):
        return any(label == 'entailment' for label in row['labels'])

    def is_correct_neutral(row):
        return all(label != 'entailment' for label in row['labels'])

    # Apply the functions and store the results in the new column 'is_label_correct'
    df['is_label_correct'] = df.apply(lambda row: is_correct_bias(row) if row['bias'] == 'bias' else is_correct_neutral(row), axis=1)
    df['predicted'] = True
    return df



,comment_text,identity_attack_x,bias,target,category,rationale,labels,probabilities,is_label_correct,predicted
156,What these femanazis fail to realize is that President Trump has hired and cut paychecks to more women blacks latinos gays lesbians transgenders and muslims than all previous presidents combined.,0.703333,bias,feminists,political,prejudice against feminists,"[contradiction, neutral, neutral, entailment, entailment, entailment, entailment, neutral, entailment, entailment, entailment, neutral, entailment, entailment, neutral]","[{'contradiction': 51.6, 'neutral': 39.2, 'entailment': 9.3}, {'contradiction': 25.6, 'neutral': 62.0, 'entailment': 12.4}, {'contradiction': 18.8, 'neutral': 60.3, 'entailment': 20.9}, {'contradiction': 2.7, 'neutral': 5.8, 'entailment': 91.5}, {'contradiction': 0.7, 'neutral': 41.8, 'entailment': 57.5}, {'contradiction': 4.4, 'neutral': 10.7, 'entailment': 84.9}, {'contradiction': 3.7, 'neutral': 24.7, 'entailment': 71.6}, {'contradiction': 21.6, 'neutral': 43.4, 'entailment': 35.0}, {'con...",True,True
36,I heard Republicans were barricaded inside polling stations with orders to shoot to kill any black hispanic poor or elderly person who looked like a Democrat who tried to vote. Only white Republicans waving the Confederate flag were allowed in to vote. That is what this story is all about. . Yup .,0.700000,bias,"republicans,blacks,hispanics","political,race",prejudice against republicans and blacks,"[contradiction, neutral, entailment, contradiction, neutral, neutral, entailment, entailment, entailment, neutral, entailment, neutral, entailment, contradiction, neutral]","[{'contradiction': 68.3, 'neutral': 30.5, 'entailment': 1.2}, {'contradiction': 4.3, 'neutral': 69.6, 'entailment': 26.1}, {'contradiction': 12.3, 'neutral': 40.1, 'entailment': 47.6}, {'contradiction': 65.6, 'neutral': 10.8, 'entailment': 23.6}, {'contradiction': 4.0, 'neutral': 60.0, 'entailment': 36.0}, {'contradiction': 17.0, 'neutral': 43.0, 'entailment': 40.0}, {'contradiction': 0.5, 'neutral': 10.2, 'entailment': 89.3}, {'contradiction': 0.4, 'neutral': 8.3, 'entailment': 91.3}, {'con...",True,True
144,I hate the GOP Senators....and every heterosexual red blooded American Male does too. They are a bunch of RINO homos. And now theyve bedded down Roy Moore,0.769231,bias,"republicans,men,homosexuals","lgbtq,gender,political","prejudice against republicans,men and homosexuals","[neutral, neutral, neutral, contradiction, neutral, entailment, neutral, neutral, entailment, neutral, neutral, neutral, entailment, entailment, entailment]","[{'contradiction': 39.5, 'neutral': 53.6, 'entailment': 6.9}, {'contradiction': 16.4, 'neutral': 66.3, 'entailment': 17.3}, {'contradiction': 3.1, 'neutral': 71.1, 'entailment': 25.8}, {'contradiction': 78.6, 'neutral': 7.6, 'entailment': 13.8}, {'contradiction': 1.1, 'neutral': 50.2, 'entailment': 48.7}, {'contradiction': 2.2, 'neutral': 8.3, 'entailment': 89.5}, {'contradiction': 2.4, 'neutral': 53.2, 'entailment': 44.4}, {'contradiction': 46.6, 'neutral': 47.8, 'entailment': 5.6}, {'contr...",True,True
54,Thats like saying the conservatives and GOP have endlessly supported racism and bigotry for the last four decades but suddenly want everyone to tolerate and embrace white supremacist. See how ridiculous that sounds.,0.500000,bias,"conservatives,whites","political,race",prejudice against conservativess,"[contradiction, neutral, neutral, contradiction, entailment, neutral, neutral, neutral, entailment, contradiction, entailment, neutral, neutral, contradiction, contradiction]","[{'contradiction': 52.0, 'neutral': 43.4, 'entailment': 4.6}, {'contradiction': 6.8, 'neutral': 57.2, 'entailment': 36.0}, {'contradiction': 21.8, 'neutral': 64.2, 'entailment': 14.0}, {'contradiction': 80.6, 'neutral': 6.5, 'entailment': 13.0}, {'contradiction': 0.7, 'neutral': 18.9, 'entailment': 80.4}, {'contradiction': 36.0, 'neutral': 53.1, 'entailment': 10.9}, {'contradiction': 6.6, 'neutral': 62.3, 'entailme